In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import xgboost as xgb
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_log_error

In [16]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')

In [17]:
def date_trans(df):
    df['Policy Start Date']= pd.to_datetime(df['Policy Start Date'])
    df['Year'] = df['Policy Start Date'].dt.year
    df['Day'] = df['Policy Start Date'].dt.day
    df['Month'] = df['Policy Start Date'].dt.month
    df['DayOfWeek'] = df['Policy Start Date'].dt.dayofweek
    df.drop('Policy Start Date' , axis =1, inplace = True)
    return df

train_df = date_trans(train_df)
test_df = date_trans(test_df)

In [18]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [19]:
# Assume train and test are already loaded as DataFrames
for col in train_df.select_dtypes(include='object').columns:
    train_df[col] = train_df[col].astype('category')
for col in test_df.select_dtypes(include='object').columns:
    test_df[col] = test_df[col].astype('category')


# Xg Boost

In [20]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer
# from sklearn.metrics import mean_squared_error, mean_squared_log_error
# from xgboost import XGBRegressor
# from numpy import log1p, expm1

# # Set random seed
# np.random.seed(42)

# # Assume train_df and test_df are your DataFrames
# df = train_df

# # Features and target
# X = df.drop('Premium Amount', axis=1)
# y = df['Premium Amount']

# # Log transform the target
# y_log = np.log1p(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned_log = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions
# test_predictions = np.zeros(len(X))  # Test predictions for each fold

# # Test set
# X_test = test_df
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # XGBoost Regressor parameters
# model_params = {'n_estimators': 7000,
#                 'learning_rate': 0.0037894924041441663,
#                 'max_depth': 9,
#                 'min_child_weight': 7,
#                 'gamma': 0.003166869962093635,
#                 'subsample': 0.8491559357878403,
#                 'colsample_bytree': 0.9931791435553496,
#                 'reg_alpha': 8.030670352805062e-08,
#                 'reg_lambda': 0.23939002451629704,
#                 "random_state": 42,
#                 'eval_metric': 'rmse',
#                 "objective": "reg:squarederror",  # XGBoost objective for regression
#                 "tree_method": "hist",  # Use GPU acceleration
#                 "device": "cuda",
#                 "enable_categorical": True  # Enable categorical handling if required
#                }

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned_log)):
#     print(f"Fold {fold + 1}/{n_splits}")
    
#     # Split data
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    
#     # Model
#     model = XGBRegressor(**model_params, early_stopping_rounds=300)
#     model.fit(X_train, y_train,
#               eval_set=[(X_valid, y_valid)],
#               verbose=0)
    
#     # Predictions (log transformed)
#     log_oof_preds = model.predict(X_valid)
    
#     # Revert log transformation
#     oof_predictions[valid_idx] = np.expm1(log_oof_preds)

#     fold_rmsle = rmsle(y.iloc[valid_idx], oof_predictions[valid_idx])
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],
#         'Actual': y.iloc[valid_idx],
#         'OOF_Pred_XGB': oof_predictions[valid_idx],
#         'Fold': fold + 1
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions
#     log_test_preds = model.predict(X_test)
#     test_preds_per_fold[:, fold] = np.expm1(log_test_preds)

# # Combine fold results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# # Output predictions
# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())

In [21]:
# sub = pd.read_csv('playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('submission.csv', index=False)
# oof_results_df.to_csv('oof_xgb.csv',index = False)
# sub.head()


# Light Boost

In [22]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_squared_error, mean_squared_log_error

# def log_transform(y):
#     """
#     Apply log transformation safely, handling zero and negative values
#     """
#     return np.log1p(y)

# def inverse_log_transform(y_log):
#     """
#     Revert log transformation
#     """
#     return np.expm1(y_log)

# # Example dataset (replace with your dataset)
# np.random.seed(42)
# df = train_df

# # Find the target column (assumes it contains 'Premium' or 'Amount')
# target_column = [col for col in df.columns if 'premium' in col.lower() or 'amount' in col.lower()]
# if not target_column:
#     raise ValueError("Could not find target column. Please specify the column name for premium/amount.")
# target_column = target_column[0]

# # Identify column types
# numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# # Remove target variable from features
# if target_column in numeric_features:
#     numeric_features.remove(target_column)
# if target_column in categorical_features:
#     categorical_features.remove(target_column)

# # Features and target
# X = df.drop(columns=[target_column])
# y = df[target_column]

# # Log transform the target variable
# y_log = log_transform(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions_log = np.zeros(len(X))  # Out-of-fold predictions in log space
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions in original space

# # Test set (replace with your actual test set)
# X_test = test_df

# # Prepare for cross-validation
# test_preds_per_fold_log = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in log space
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in original space
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # LightGBM Regressor parameters
# model_params = {
#     'objective': 'regression',
#     'metric': 'rmsle',
#     'random_state': 42,
#     'is_unbalance': True,
#       'nan_as_missing': True,
#       'n_estimators': 2900, 
#       'learning_rate': 0.009292502487271825,
#       'max_depth': 11,
#       'num_leaves': 88, 
#       'min_child_samples': 46,
#       'min_child_weight': 0.0008888702484611456, 
#       'subsample': 0.8552895795397466, 
#       'subsample_freq': 5, 
#       'colsample_bytree': 0.9135439280459978,
#       'reg_alpha': 0.0021100979917979567,
#       'reg_lambda': 5.0153235478869446e-08,
#       'verbose': -1,
#         'device': 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 0,
#         'gpu_use_dp': True  
# }

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned)):
#     print(f"Fold {fold + 1}/{n_splits}")
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train_log, y_valid_log = y_log.iloc[train_idx], y_log.iloc[valid_idx]
#     y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
#     # Model with early stopping on log-transformed target
#     model = LGBMRegressor(**{**model_params, 'verbose': -1})
#     model.fit(X_train, y_train_log, 
#               eval_set=[(X_valid, y_valid_log)],
#               )
    
#     # Predictions in log space
#     oof_predictions_log[valid_idx] = model.predict(X_valid)
    
#     # Convert log predictions back to original space
#     oof_predictions[valid_idx] = inverse_log_transform(oof_predictions_log[valid_idx])

#     fold_rmsle = rmsle(y.iloc[valid_idx], oof_predictions[valid_idx])
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results with ID, target, and OOF predictions
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],       # Assuming X has an index as IDs
#         'Actual': y.iloc[valid_idx],    # Actual target values
#         'OOF_Pred_LGB': oof_predictions[valid_idx],    # OOF predictions
#         'Fold': fold + 1               # Fold number
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions for this fold
#     test_preds_per_fold_log[:, fold] = model.predict(X_test)
#     test_preds_per_fold[:, fold] = inverse_log_transform(test_preds_per_fold_log[:, fold])

# # Combine OOF results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())

In [23]:
# sub = pd.read_csv('playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('submission.csv', index=False)
# sub.head()
# oof_results_df.to_csv('oof_lgbm.csv',index = False)

# Cat Boost

In [24]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error, mean_squared_log_error

# def log_transform(y):
#     """
#     Apply log transformation safely, handling zero and negative values
#     """
#     return np.log1p(y)

# def inverse_log_transform(y_log):
#     """
#     Revert log transformation
#     """
#     return np.expm1(y_log)

# # Example dataset (replace with your dataset)
# np.random.seed(42)
# df = train_df

# # Find the target column (assumes it contains 'Premium' or 'Amount')
# target_column = [col for col in df.columns if 'premium' in col.lower() or 'amount' in col.lower()]
# if not target_column:
#     raise ValueError("Could not find target column. Please specify the column name for premium/amount.")
# target_column = target_column[0]

# # Identify column types
# numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# # Remove target variable from features
# if target_column in numeric_features:
#     numeric_features.remove(target_column)
# if target_column in categorical_features:
#     categorical_features.remove(target_column)

# # Features and target
# X = df.drop(columns=[target_column])
# y = df[target_column]

# # Log transform the target variable
# y_log = log_transform(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions_log = np.zeros(len(X))  # Out-of-fold predictions in log space
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions in original space

# # Test set (replace with your actual test set)
# X_test = test_df

# # Prepare for cross-validation
# test_preds_per_fold_log = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in log space
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in original space
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # CatBoost Regressor parameters
# model_params = {
#     'loss_function': 'RMSE',
#     'eval_metric': 'RMSLE',
#     'random_seed': 42,
#     'iterations': 2900,
#     'learning_rate': 0.009292502487271825,
#     'depth': 11,
#     'l2_leaf_reg': 3.5,
#     'subsample': 0.8552895795397466,
#     'colsample_bylevel': 0.9135439280459978,
#     'bootstrap_type': 'Bernoulli',
#     'verbose': False,
#     'task_type': 'GPU',
#     'devices': '0'
# }

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned)):
#     print(f"Fold {fold + 1}/{n_splits}")
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train_log, y_valid_log = y_log.iloc[train_idx], y_log.iloc[valid_idx]
#     y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
#     # Model with early stopping on log-transformed target
#     model = CatBoostRegressor(**model_params)
#     model.fit(X_train, y_train_log, 
#               eval_set=(X_valid, y_valid_log),
#               use_best_model=True)
    
#     # Predictions in log space
#     oof_predictions_log[valid_idx] = model.predict(X_valid)
    
#     # Convert log predictions back to original space
#     oof_predictions[valid_idx] = inverse_log_transform(oof_predictions_log[valid_idx])

#     fold_rmsle = np.sqrt(mean_squared_log_error(y.iloc[valid_idx], oof_predictions[valid_idx]))
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results with ID, target, and OOF predictions
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],       # Assuming X has an index as IDs
#         'Actual': y.iloc[valid_idx],    # Actual target values
#         'OOF_Pred_CatBoost': oof_predictions[valid_idx],    # OOF predictions
#         'Fold': fold + 1               # Fold number
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions for this fold
#     test_preds_per_fold_log[:, fold] = model.predict(X_test)
#     test_preds_per_fold[:, fold] = inverse_log_transform(test_preds_per_fold_log[:, fold])

# # Combine OOF results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())


# Linear Regression

In [25]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, mean_squared_log_error
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer

# def log_transform(y):
#     """
#     Apply log transformation safely, handling zero and negative values
#     """
#     return np.log1p(y)

# def inverse_log_transform(y_log):
#     """
#     Revert log transformation
#     """
#     return np.expm1(y_log)

# def rmsle(y_true, y_pred):
#     """
#     Calculate Root Mean Squared Logarithmic Error
#     """
#     return np.sqrt(mean_squared_log_error(y_true, y_pred))

# # Example dataset (replace with your dataset)
# np.random.seed(42)
# df = train_df

# # Find the target column (assumes it contains 'Premium' or 'Amount')
# target_column = [col for col in df.columns if 'premium' in col.lower() or 'amount' in col.lower()]
# if not target_column:
#     raise ValueError("Could not find target column. Please specify the column name for premium/amount.")
# target_column = target_column[0]

# # Identify column types
# numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# # Remove target variable from features
# if target_column in numeric_features:
#     numeric_features.remove(target_column)
# if target_column in categorical_features:
#     categorical_features.remove(target_column)

# # Features and target
# X = df.drop(columns=[target_column])
# y = df[target_column]

# # Log transform the target variable
# y_log = log_transform(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Test set (replace with your actual test set)
# X_test = test_df

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', SimpleImputer(strategy='median'), numeric_features),
#         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
#     ]
# )

# # LinearRegression model parameters (no hyperparameters to tune)
# model = LinearRegression()

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions_log = np.zeros(len(X))  # Out-of-fold predictions in log space
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions in original space

# # Prepare for cross-validation
# test_preds_per_fold_log = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in log space
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in original space
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned)):
#     print(f"Fold {fold + 1}/{n_splits}")
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train_log, y_valid_log = y_log.iloc[train_idx], y_log.iloc[valid_idx]
#     y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
#     # Model pipeline with preprocessing and regressor
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', model)
#     ])
    
#     # Fit the model
#     pipeline.fit(X_train, y_train_log)
    
#     # Predictions in log space
#     oof_predictions_log[valid_idx] = pipeline.predict(X_valid)
    
#     # Convert log predictions back to original space
#     oof_predictions[valid_idx] = inverse_log_transform(oof_predictions_log[valid_idx])

#     fold_rmsle = rmsle(y.iloc[valid_idx], oof_predictions[valid_idx])
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results with ID, target, and OOF predictions
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],       # Assuming X has an index as IDs
#         'Actual': y.iloc[valid_idx],    # Actual target values
#         'OOF_Pred_LR': oof_predictions[valid_idx],    # OOF predictions
#         'Fold': fold + 1               # Fold number
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions for this fold
#     test_preds_per_fold_log[:, fold] = pipeline.predict(X_test)
#     test_preds_per_fold[:, fold] = inverse_log_transform(test_preds_per_fold_log[:, fold])

# # Combine OOF results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())


# sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('lrsubmission.csv', index=False)
# sub.head()
# oof_results_df.to_csv('oof_lr.csv',index = False)
# sub.head()

In [26]:
# sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('lrsubmission.csv', index=False)
# sub.head()
# oof_results_df.to_csv('oof_lr.csv',index = False)
# sub.head()

# Aada Boost

In [27]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
# from sklearn.metrics import mean_squared_error, mean_squared_log_error
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# # Set random seed
# np.random.seed(42)

# # Assume train_df and test_df are your DataFrames
# df = train_df

# # Features and target
# X = df.drop('Premium Amount', axis=1)
# y = df['Premium Amount']

# # Log transform the target
# y_log = np.log1p(y)

# # Stratify target by binning into discrete intervals
# binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# y_binned_log = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# # Identify categorical and numerical columns
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# # Create imputers for numerical and categorical columns
# numerical_imputer = SimpleImputer(strategy='mean')  # Use mean for numerical features
# categorical_imputer = SimpleImputer(strategy='most_frequent')  # Use most frequent for categorical features
# categorical_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)  # One hot encoding for categorical features

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_imputer, numerical_cols),
#         ('cat', Pipeline([
#             ('imputer', categorical_imputer),
#             ('encoder', categorical_encoder)
#         ]), categorical_cols)
#     ]
# )

# # Stratified K-Fold
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Arrays to store predictions
# oof_predictions = np.zeros(len(X))  # Out-of-fold predictions
# test_predictions = np.zeros(len(X))  # Test predictions for each fold

# # Test set
# X_test = test_df
# test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold
# fold_oof_results = []  # To store ID, target, and OOF predictions

# # Cross-validation loop
# for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned_log)):
#     print(f"Fold {fold + 1}/{n_splits}")
    
#     # Split data
#     X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    
#     # Apply preprocessing to both train and valid sets
#     X_train_transformed = preprocessor.fit_transform(X_train)
#     X_valid_transformed = preprocessor.transform(X_valid)
    
#     # Model
#     model = AdaBoostRegressor(
#         base_estimator=DecisionTreeRegressor(max_depth=5),
#         n_estimators=100,
#         learning_rate=0.1,
#         random_state=42
#     )
#     model.fit(X_train_transformed, y_train)
    
#     # Predictions (log transformed)
#     log_oof_preds = model.predict(X_valid_transformed)
    
#     # Revert log transformation
#     oof_predictions[valid_idx] = np.expm1(log_oof_preds)

#     fold_rmsle = np.sqrt(mean_squared_log_error(y.iloc[valid_idx], oof_predictions[valid_idx]))
#     print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
#     # Store fold results
#     fold_result = pd.DataFrame({
#         'ID': X.index[valid_idx],
#         'Actual': y.iloc[valid_idx],
#         'OOF_Pred': oof_predictions[valid_idx],
#         'Fold': fold + 1
#     })
#     fold_oof_results.append(fold_result)
    
#     # Test set predictions (transform test data)
#     X_test_transformed = preprocessor.transform(X_test)
#     log_test_preds = model.predict(X_test_transformed)
#     test_preds_per_fold[:, fold] = np.expm1(log_test_preds)

# # Combine fold results
# oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# # Average predictions on test data
# final_test_predictions = test_preds_per_fold.mean(axis=1)

# # Evaluate OOF predictions
# oof_mse = mean_squared_error(y, oof_predictions)
# oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

# print(f"OOF Mean Squared Error: {oof_mse:.4f}")
# print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# # Output predictions
# print("Final Test Predictions:", final_test_predictions)
# print(oof_results_df.head())


In [28]:
# sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = final_test_predictions
# sub.to_csv('submission.csv', index=False)
# oof_results_df.to_csv('oof_aaba.csv',index = False)
# sub.head()


# Ensembling

In [ ]:
gydhsgkjdf